## 计算召回率

In [8]:
import torch
from torch import optim
from torch import nn
import numpy as np
from load_data_224 import image_datasets, dataloaders
import my_model
from torchvision import models

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            # 把模型的所有层的参数的 requires_grad 属性设置为 false，表示不再对原本的参数进行训练
            param.requires_grad = False

In [10]:
# 初始化模型
def initialize_model(model_name, num_classes=3, feature_extract=False, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == 'myResnet':
        model_ft = my_model.ResNet()

        input_size = 224
    elif model_name == 'resnet34':
        model_ft = models.resnet34(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes),
                                    nn.LogSoftmax(dim=1))
        input_size = 224
    elif model_name == 'resnet101':
        model_ft = models.resnet101(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes),
                                    nn.LogSoftmax(dim=1))
        input_size = 224
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#加载模型
model, _ = initialize_model('resnet101')
model.to(device)
# 加载checkpoint文件
directory = 'checkpoint_resnet101_1_30'
filename = 'checkpoint.pth'

checkpoint = torch.load(directory+'/'+filename)
model.load_state_dict(checkpoint['state_dict'])

model.eval()

a:\WorkApps\anaconda3\envs\deeplearning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# 获取类别名称列表  
classes = image_datasets['valid'].classes  

# 初始化一个字典来存储每个类别的数量  
class_to_count = {cls: 0 for cls in classes} 
print(class_to_count)

# 初始化一个字典来存储每个类别真正例（TP）的数量
true_positives = {cls: 0 for cls in classes}
print(true_positives)



{'COVID19': 0, 'NORMAL': 0, 'PNEUMONIA': 0}
{'COVID19': 0, 'NORMAL': 0, 'PNEUMONIA': 0}


In [13]:
print('类别名称映射到其对应的索引：', image_datasets['valid'].class_to_idx)

# 计算验证集中各个分类的召回率
for inputs, labels in dataloaders['valid']:
    output = model(inputs.cuda())
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(preds_tensor.cpu().numpy())
    # print(preds)
    # print(labels.numpy())
    # print(len(labels))
    # 计算召回率：A_召回率 = (A的真正例)/(A的真正例+A的假正例) = (正确预测为A的个数)/(A类别的总数)
    for index in range(len(labels)):
        if labels[index] == 0:
            class_to_count['COVID19'] += 1
            if labels[index] == preds[index]:
                true_positives['COVID19'] += 1
        if labels[index] == 1:
            class_to_count['NORMAL'] += 1
            if labels[index] == preds[index]:
                true_positives['NORMAL'] += 1
        if labels[index] == 2:
            class_to_count['PNEUMONIA'] += 1
            if labels[index] == preds[index]:
                true_positives['PNEUMONIA'] += 1

print(class_to_count)
print(true_positives)

类别名称映射到其对应的索引： {'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2}
{'COVID19': 116, 'NORMAL': 317, 'PNEUMONIA': 855}
{'COVID19': 111, 'NORMAL': 295, 'PNEUMONIA': 826}


In [14]:
recall_COVID19 = true_positives['COVID19'] / class_to_count['COVID19']
print(recall_COVID19)
recall_NORMAL = true_positives['NORMAL'] / class_to_count['NORMAL']
print(recall_NORMAL)
recall_PNEUMONIA = true_positives['PNEUMONIA'] / class_to_count['PNEUMONIA']
print(recall_PNEUMONIA)

0.9568965517241379
0.9305993690851735
0.9660818713450292
